In [ ]:
#gameId = "doapp"
gameId = "sfiii3n"
#gameId = "umk3"
#gameId = "tektagt"

In [ ]:
import sys, os
from os.path import expanduser 
import time
import cv2
import numpy as np

homeDir = expanduser("~") 

timeDepSeed = int((time.time()-int(time.time()-0.5))*1000)

sys.path.append(os.path.join(os.path.abspath(''), '../.'))  
sys.path.append(os.path.join(os.path.abspath(''), '../../gym/'))
                                                                                                                                 
from utils import discreteToMultiDiscreteAction
from randomTestSBWrappers import showObs
from makeStableBaselinesEnv import makeStableBaselinesEnv

In [ ]:
# Common settings
diambraKwargs = {}
diambraKwargs["romsPath"] = "../../roms/mame/"
diambraKwargs["binaryPath"] = "../../customMAME/"
diambraKwargs["frameRatio"] = 6
diambraKwargs["throttle"] = False
diambraKwargs["sound"] = diambraKwargs["throttle"]

diambraKwargs["player"] = "Random" # 1P
#diambraKwargs["player"] = "P1P2" # 2P

In [ ]:
if gameId != "tektagt":
    diambraKwargs["characters"] = ["Random", "Random"]
else:
    diambraKwargs["characters"] = [["Random", "Random"], ["Random", "Random"]]
diambraKwargs["charOutfits"] = [2, 2]

In [ ]:
# DIAMBRA gym kwargs
diambraGymKwargs = {}
diambraGymKwargs["actionSpace"] = ["multiDiscrete", "multiDiscrete"]
diambraGymKwargs["attackButCombinations"] = [False, False]
diambraGymKwargs["actBufLen"] = 12
if diambraKwargs["player"] != "P1P2":
    diambraGymKwargs["showFinal"] = False
    diambraGymKwargs["continueGame"] = -1.0
    diambraGymKwargs["actionSpace"] = diambraGymKwargs["actionSpace"][0]
    diambraGymKwargs["attackButCombinations"] = diambraGymKwargs["attackButCombinations"][0]
    
idxList = [0, 1]
if diambraKwargs["player"] != "P1P2":
    idxList = [0]

In [ ]:
# Recording kwargs
trajRecKwargs = {}                                                          
trajRecKwargs["userName"] = "Alex"
trajRecKwargs["filePath"] = os.path.join( homeDir, "DIAMBRA/trajRecordings", gameId)
trajRecKwargs["ignoreP2"] = 0                                    
trajRecKwargs["commitHash"] = "0000000"

In [ ]:
trajRecKwargs = None

In [ ]:
# Env wrappers kwargs
wrapperKwargs = {}
wrapperKwargs["noOpMax"] = 0
wrapperKwargs["hwcObsResize"] = [128, 128, 1]
wrapperKwargs["normalizeRewards"] = True
wrapperKwargs["clipRewards"] = False
wrapperKwargs["frameStack"] = 4
wrapperKwargs["dilation"] = 1
wrapperKwargs["scale"] = True
wrapperKwargs["scaleMod"] = 0

In [ ]:
# Additional obs key list
keyToAdd = []
keyToAdd.append("actionsBuf") # env.actBufLen*(env.n_actions[0]+env.n_actions[1])

if gameId != "tektagt":                                                         
    keyToAdd.append("ownHealth")   # 1                                            
    keyToAdd.append("oppHealth")   # 1                                                
    if gameId == "sfiii3n":
        keyToAdd.append("ownSuperBarVal")   # 1                                                
        keyToAdd.append("ownSuperCount")    # 1                                                
        keyToAdd.append("ownSuperMaxCount") # 1                                                
        keyToAdd.append("ownSuperSelected")  # 1
        keyToAdd.append("ownStunBar")   # 1
        keyToAdd.append("ownStunned")   # 1
        keyToAdd.append("oppSuperBarVal")   # 1                                                
        keyToAdd.append("oppSuperCount")    # 1                                                
        keyToAdd.append("oppSuperMaxCount") # 1                                                
        keyToAdd.append("oppSuperSelected")  # 1
        keyToAdd.append("oppStunBar")   # 1
        keyToAdd.append("oppStunned")   # 1
else:                                                                           
    keyToAdd.append("ownHealth1") # 1                                             
    keyToAdd.append("ownHealth2") # 1                                             
    keyToAdd.append("oppHealth1") # 1                                              
    keyToAdd.append("oppHealth2") # 1  
    keyToAdd.append("ownActiveChar") # 1
    keyToAdd.append("oppActiveChar") # 1
    
keyToAdd.append("ownPosition")     # 1
keyToAdd.append("oppPosition")     # 1
if diambraKwargs["player"] != "P1P2":
    keyToAdd.append("stage")           # 1
keyToAdd.append("ownChar")       # len(env.charNames)
keyToAdd.append("oppChar")       # len(env.charNames)

In [ ]:
hardCore = False
envId = gameId + "_randomTestSBWrap"
numOfEnvs = 1
env = makeStableBaselinesEnv(envId, numOfEnvs, timeDepSeed, diambraKwargs, diambraGymKwargs, 
                             wrapperKwargs, trajRecKwargs, keyToAdd=keyToAdd, noVec=True, hardCore=hardCore)

In [ ]:
print("Observation Space:", env.observation_space)
print("Action Space:", env.action_space)
if not hardCore:
    print("Keys to Dict:")
    for k,v in env.keysToDict.items():
        print(k, v)

In [ ]:
limAct = [None, None]
if diambraKwargs["player"] != "P1P2":
    for idx in range(2):                                                        
        limAct[idx] = [env.actBufLen * env.nActions[0],                                
                       env.actBufLen * env.nActions[0] + env.actBufLen * env.nActions[1]]     
else:
    for idx in range(2):                                                        
        limAct[idx] = [env.actBufLen * env.nActions[idx][0],                                
                       env.actBufLen * env.nActions[idx][0] + env.actBufLen * env.nActions[idx][1]]     

In [ ]:
actionsPrintDict = env.printActionsDict()
observation = env.reset()

In [ ]:
showObs(observation, gameId, env.actBufLen, 0, True, env.charNames, limAct, hardCore, idxList)

In [ ]:
cumulativeEpRew = 0.0
cumulativeEpRewAll = []

maxNumEp = 100
currNumEp = 0

while currNumEp < maxNumEp:

    actions = [None, None]
    if diambraKwargs["player"] != "P1P2":
        actions = env.action_space.sample()
        
        if diambraGymKwargs["actionSpace"] == "discrete":
            moveAction, attAction = discreteToMultiDiscreteAction(actions, env.nActions[0])
        else:
            moveAction, attAction = actions[0], actions[1]
            
        print("(P1) {} {}".format(actionsPrintDict[0][moveAction],       
                                  actionsPrintDict[1][attAction])) 
        
    else:
        
        for idx in range(2):
        
            actions[idx] = env.action_space["P{}".format(idx+1)].sample()

            if diambraGymKwargs["actionSpace"][idx] == "discrete":
                moveAction, attAction = discreteToMultiDiscreteAction(actions[idx], env.nActions[idx][0])
            else:
                moveAction, attAction = actions[idx][0], actions[idx][1]
        
            if diambraKwargs["player"] != "P1P2" and idx == 1:
                continue
            
            print("(P{}) {} {}".format(idx+1, actionsPrintDict[0][moveAction],       
                                              actionsPrintDict[1][attAction])) 
        
    if diambraKwargs["player"] == "P1P2" or diambraGymKwargs["actionSpace"] != "discrete":
        actions = np.append(actions[0], actions[1])   
    
    observation, reward, done, info = env.step(actions)
    
    print("action = ", actions)
    print("reward:", reward)
    print("done = ", done)
    for k, v in info.items():
        print("info[\"{}\"] = {}".format(k, v))
    showObs(observation, gameId, env.actBufLen, 0, True, env.charNames, limAct, hardCore, idxList)
        
    print("----------")
    
    cumulativeEpRew += reward
    
    if np.any(done):
        currNumEp += 1
        print("Ep. # = ", currNumEp)
        print("Ep. Cumulative Rew # = ", cumulativeEpRew)
        cumulativeEpRewAll.append(cumulativeEpRew)
        cumulativeEpRew = 0.0

        observation = env.reset()
        showObs(observation, gameId, env.actBufLen, 0, True, env.charNames, limAct, hardCore, idxList)

print("Mean cumulative reward = ", np.mean(cumulativeEpRewAll))    
print("Std cumulative reward = ", np.std(cumulativeEpRewAll))       
    
env.close()